# 14 基于mindspore实现二分类损失函数

损失函数，亦称目标函数，用于衡量预测值与真实值差异的程度。

在深度学习中，模型训练就是通过不断迭代来缩小损失函数值的过程。因此，在模型训练过程中损失函数的选择非常重要，一个好的损失函数能有效提升模型的性能。

mindspore.nn模块中提供了许多通用损失函数，但这些通用损失函数无法满足所有需求，很多情况需要用户自定义所需的损失函数。因此，本课程介绍如何自定义损失函数。

# 14.1 内置损失函数

首先介绍 mindspore. $\mathrm{nn}$ 模块中内置的损失函数。
下例以 $n n$. L1Loss 为例，计算预恻值和目标值之间的平均绝对误差:
$$
\ell(x, y)=L=\left\{l_1, \ldots, l_N\right\}^{\top}, \quad \text { with } l_n=\left|x_n-y_n\right|
$$
其中N为数据集中的 batch_size 值。
$$
\ell(x, y)= \begin{cases}\operatorname{mean}(L), & \text { if reduction }=\text { 'mean'; } \\ \operatorname{sum}(L), & \text { if reduction='sum' }\end{cases}
$$
nn.L1Loss 中的参数 reduction 取值可为 mean， sum，或 none 。若 reduction 为 mean 或 sum，则输出一个经过均值或求和后的标量 Tensor (降维) ；若 reduction 为 none，则所输出Tensor的shape为广播后的shape。

In [13]:
import numpy as np
import mindspore.nn as nn
import mindspore as ms

    '''
    loss：输出loss均值
    loss_sum：输出loss和
    loss_none：输出loss原值
    '''


input_data = ms.Tensor(np.array([1, 0, 1, 0, 1, 0]).astype(np.float32))
target_data = ms.Tensor(np.array([0, 0, 1, 1, 1, 0]).astype(np.float32))

print("loss:", loss(input_data, target_data))
print("loss_sum:", loss_sum(input_data, target_data))
print("loss_none:\n", loss_none(input_data, target_data))

loss: 0.33333334
loss_sum: 2.0
loss_none:
 [1. 0. 0. 1. 0. 0.]


In [ ]:
#参考答案
# 输出loss均值
loss = nn.L1Loss()
# 输出loss和
loss_sum = nn.L1Loss(reduction='sum')
# 输出loss原值
loss_none = nn.L1Loss(reduction='none')

自定义损失函数的方法有两种: 一是基于 $n n$.Cell 来定义损失函数；二是 nn.LossBase 来定义损失函数。 nn.LossBase 继承自 nn.Ce11，额 外提供了 get_loss 方法，利用 reduction 参数对损失值求和或求均值，输出一个标量。
下面将分别使用继承 Cell 和继承 LossBase 的方法，来定义平均绝对误差损失函数(Mean Absolute Error， MAE)，MAE算法的公式如下所 示:
$$
\text { loss }=\frac{1}{m} \sum_{i=1}^m\left|y_i-f\left(x_i\right)\right|
$$
上式中 $f(x)$ 为预测值， $y$ 为样本真实值，loss 为预测值与真实值之间距离的平均值。

# 14.2 基于nn.Cell构造损失函数

nn.Cell 是MindSpore的基类，不但可用于构建网络，还可用于定义损失函数。使用 $n n$.Cel1 定义损失函数的过程与定义一个普通的网络 相似，差别在于，其执行逻辑部分要计算的是前向网络输出与真实值之间的误差。

In [14]:
import mindspore.ops as ops

class MAELoss(nn.Cell):
    """自定义损失函数MAELoss"""

    def __init__(self):
    '''
    初始化

    '''
    def construct(self, base, target):

    '''
    调用算子
    '''

loss = MAELoss()

input_data = ms.Tensor(np.array([1, 0, 1, 0, 1, 0]).astype(np.float32))  # 生成预测值
target_data = ms.Tensor(np.array([0, 0, 1, 1, 1, 0]).astype(np.float32)) # 生成真实值

output = loss(input_data, target_data)
print(output)

0.33333334


In [ ]:
#参考答案
    def __init__(self):
        """初始化"""
        super(MAELoss, self).__init__()
        self.abs = ops.Abs()
        self.reduce_mean = ops.ReduceMean()


In [ ]:
#参考答案
    def construct(self, base, target):
        """调用算子"""
        x = self.abs(base - target)
        return self.reduce_mean(x)

# 14.3 基于nn.LossBase构造损失函数

基于nn.LossBase构造损失函数MAELoss与基于nn.Cell构造损失函数的过程类似，都要重写__init__方法和construct方法。

nn.LossBase可使用方法get_loss将reduction应用于损失计算。

In [15]:
class MAELoss(nn.LossBase):
    """自定义损失函数MAELoss"""

    def __init__(self, reduction="mean"):
    '''
    初始化

    '''
    def construct(self, base, target):
    '''
    返回loss均值

    '''
loss = MAELoss()

input_data = ms.Tensor(np.array([1, 0, 1, 0, 1, 0]).astype(np.float32))  # 生成预测值
target_data = ms.Tensor(np.array([0, 0, 1, 1, 1, 0]).astype(np.float32))  # 生成真实值

output = loss(input_data, target_data)
print(output)

0.33333334


In [ ]:
#参考答案
    def __init__(self, reduction="mean"):
        """初始化并求loss均值"""
        super(MAELoss, self).__init__(reduction)
        self.abs = ops.Abs()  # 求绝对值算子

In [ ]:
#参考答案
    def construct(self, base, target):
        x = self.abs(base - target)
        return self.get_loss(x)  # 返回loss均值

# 14.4 损失函数与模型训练

损失函数MAELoss自定义完成后，可使用MindSpore的接口Model中train接口进行模型训练，构造Model时需传入前向网络、损失函数和优化器，Model会在内部将它们关联起来，生成一个可用于训练的网络模型。

在Model中，前向网络和损失函数通过nn.WithLossCell关联起来，nn.WithLossCell支持两个输入，分别为data和label。

In [16]:

import mindspore as ms
from mindspore import dataset as ds
from mindspore.common.initializer import Normal
from mindvision.engine.callback import LossMonitor

def get_data(num, w=2.0, b=3.0):
    """生成数据及对应标签"""
    for _ in range(num):
        x = np.random.uniform(-10.0, 10.0)
        noise = np.random.normal(0, 1)
        y = x * w + b + noise
        yield np.array([x]).astype(np.float32), np.array([y]).astype(np.float32)

def create_dataset(num_data, batch_size=16):
    """加载数据集"""
    dataset = ds.GeneratorDataset(list(get_data(num_data)), column_names=['data', 'label'])
    dataset = dataset.batch(batch_size)
    return dataset

class LinearNet(nn.Cell):
    '''
    定义线性回归网络

    '''

ds_train = create_dataset(num_data=160)
net = LinearNet()
loss = MAELoss()
opt = nn.Momentum(net.trainable_params(), learning_rate=0.005, momentum=0.9)

# 使用model接口将网络、损失函数和优化器关联起来
model = ms.Model(net, loss, opt)
model.train(epoch=1, train_dataset=ds_train, callbacks=[LossMonitor(0.005)])


[WARNING] ME(9968:9256,MainProcess):2022-12-23-15:17:50.302.874 [mindspore\dataset\engine\datasets_user_defined.py:656] Python multiprocessing is not supported on Windows platform.


Epoch:[  0/  1], step:[    1/   10], loss:[9.476/9.476], time:1567.807 ms, lr:0.00500
Epoch:[  0/  1], step:[    2/   10], loss:[11.154/10.315], time:7.977 ms, lr:0.00500
Epoch:[  0/  1], step:[    3/   10], loss:[11.116/10.582], time:7.981 ms, lr:0.00500
Epoch:[  0/  1], step:[    4/   10], loss:[9.329/10.269], time:1.994 ms, lr:0.00500
Epoch:[  0/  1], step:[    5/   10], loss:[9.983/10.212], time:3.992 ms, lr:0.00500
Epoch:[  0/  1], step:[    6/   10], loss:[10.381/10.240], time:1.994 ms, lr:0.00500
Epoch:[  0/  1], step:[    7/   10], loss:[6.865/9.758], time:14.958 ms, lr:0.00500
Epoch:[  0/  1], step:[    8/   10], loss:[7.894/9.525], time:0.995 ms, lr:0.00500
Epoch:[  0/  1], step:[    9/   10], loss:[6.326/9.169], time:0.997 ms, lr:0.00500
Epoch:[  0/  1], step:[   10/   10], loss:[6.688/8.921], time:0.997 ms, lr:0.00500
Epoch time: 1692.437 ms, per step time: 169.244 ms, avg loss: 8.921


In [ ]:
#参考答案
"""定义线性回归网络"""
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc = nn.Dense(1, 1, Normal(0.02), Normal(0.02))

    def construct(self, x):
        return self.fc(x)